# Ejercicios propuestos: *Input*/*Output* + Serialización + RegEx

Los siguientes problemas se dejan como opción para ejercitar los conceptos revisados sobre I/O, serialización y RegEx (semana 11). Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una issue en el foro del curso.

## Ejercicio 1: `bytearrays` y *context managers*

El archivo **simpatico.bmp** ha sido corrompido con *bytes* no correspondientes. Para limpiarlo, deberás implementar el siguiente algoritmo:

1. Abrir el archivo **malvado.mal** en modo lectura de *bytes*.
2. Obtener *chunks* de 8 bytes.
3. Encontrar el máximo de esos 8 *bytes*.
4. Eliminar todos los *bytes* con ese valor dentro del *chunk*.
5. Repetir el proceso con el resto de los *bytes*.
6. Escribir el archivo **simpatico.bmp** con los bytes obtenidos.

**Sobre los pasos 2 a 4**: 

- Considerando un *chunk* de 8 *bytes*, con los siguientes valores (paso 2): `4 8 255 15 16 23 255 42`
- Se obtiene el máximo de ese *chunk* de bytes (paso 3): `255`.
- Se eliminan todos los *bytes* con ese valor, resultando: `4 8 15 16 23 42`

In [1]:
# Se abre el archivo
with open('malvado.mal', 'rb') as bytefile:
    byte_read = bytefile.read()
    byte_out = bytearray()
    # Se recorre su contenido en chunks de 8 bytes
    for i in range(0, len(byte_read), 8):
        # Se obtiene un chunk de 8 bytes
        chunk = bytearray(byte_read[i:i+8])
        # Se obtiene el valor más alto
        maximum = max(chunk)
        # Se eliminan todos los bytes con ese valor
        while maximum in chunk:
            chunk.remove(maximum)
        byte_out += chunk

# Se escribe el nuevo archivo
with open('simpatico.bmp', 'wb') as file_out:
    file_out.write(byte_out)

## Ejercicio 2: Serialización con JSON  y *f-strings*

¡Han contaminado el recetario de las pizzas del Tini Tamburini! Es por esta razón que han venido a pedirte ayuda mediante este ejercicio propuesto.

Deberás deserializar el archivo **pizzeria.json** que contiene una lista de objetos de clase `Pizza`. Estos objetos tienen el atributo `ingredientes`, el cual ha sido contaminado con elementos que no son comestibles. Para identificarlos, el Tini le entrega un archivo llamado **comestibles.json**, el cual será de ayuda para filtrar los ingredientes mediante el uso un `object_hook`. Finalmente, imprime todas las pizzas utilizando *f-strings*.

In [2]:
import json


class Pizza:
    def __init__(self, nombre, ingredientes, *args, **kwargs):
        self.nombre = nombre
        self.ingredientes = ingredientes
    def __repr__(self):
        return self.nombre
    def __str__(self):
        return f"Pizza: {self.nombre}\nIngredientes: {self.ingredientes}"

    
def filtrar_comestibles(diccionario):
    # Aquí deberás cargar la lista de comestibles y quitar del atributo
    # correspondiente aquellos elementos que no lo sean
    with open('comestibles.json') as comestibles_file:
        comestible = json.load(comestibles_file)
    diccionario['ingredientes'] = list(filter(lambda x: x in comestible, 
                                              diccionario['ingredientes']))
    return Pizza(**diccionario)


# Carga las pizzas del archivo pizzeria.json
with open('pizzeria.json') as archivo_pizzas:
    lista_pizzas = json.load(archivo_pizzas, object_hook=filtrar_comestibles)

# y finalmente imprime la lista de pizzas usando f-strings
for pizza in lista_pizzas:
    print(pizza) # Se usa f-strings en el método __str__ de Pizza
    print()

Pizza: hawaiana
Ingredientes: ['pinia', 'queso', 'jamon']

Pizza: vegetariana
Ingredientes: ['queso', 'aceitunas', 'champiniones']

Pizza: machas a la parmesana
Ingredientes: ['machas', 'queso']

Pizza: pepperoni
Ingredientes: ['pepperoni', 'queso']

Pizza: pollo bbq
Ingredientes: ['cebolla', 'queso', 'pollo bbq']

Pizza: jamon palmito
Ingredientes: ['jamon', 'palmitos', 'queso']

Pizza: white
Ingredientes: ['aceitunas', 'tomate']

Pizza: napolitana
Ingredientes: ['queso', 'albahaca', 'tomate']

Pizza: queso
Ingredientes: ['queso', 'queso']



## Ejercicio 3: Serialización con JSON y *pickle*

Mientras estás de visita en un museo, se descubre que en las descripciones de un conjunto de obras de arte se encuentra un mensaje oculto, el cual solo se puede descifrar a través del uso de *pickle* para serialización. Es por esto, que el director del museo pide tu ayuda.

Para lograr descifrar el mensaje oculto, deberás implementar las siguientes funciones:

- `cargar_obras(ruta_obras)`: El archivo **operas.json** contiene toda la información de las obras. Las obras contienen algunos datos que no son relevantes para la solución de este misterio. Por esto, se te entrega el archivo **caratteristicas.json**, con las características que sí deberás obtener de las obras, mediante la función `obras_hook`, que deberá retornar una **lista de objetos de la clase `Obra`**.
- `generar_mensaje(lista_obras)`: Esta función debe serializar cada una de las obras utilizando `pickle`. Durante este proceso, deberás agregar a cada obra el attributo `messaggio`, con un mensaje generado por 3 palabras al azar del atributo `descrizione`. Los archivos generados deben ser guardados en la carpeta **Obras** con el siguiente formato: `<nome>-<autore>.opera` (sin `<` ni `>`), donde `nome` y `autore` son el nombre de la obra y el nombre de su respectivo autor. 

In [3]:
import json
import os
import pickle
import random


CARATTERISTICAS_CAMMINO = 'caratteristicas.json'
OPERAS_CAMMINO = 'operas.json'

class Obra:
    """
    Clase para las Obras de Antonini Da Ossa
    """

    def __init__(self, nome=None, autore=None, anno=None, posto=None,
                 stile=None, descrizione=None):

        self.nome = nome
        self.autore = autore
        self.anno = anno
        self.posto = posto
        self.stile = stile
        self.descrizione = descrizione

    def __getstate__(self):
        """
        Serializa las obras, agregando el atributo 'messaggio' al diccionario.
        """
        # Se crea el 'messaggio'
        messaggio = " ".join(random.sample(self.descrizione.split(), 3))
        # Se copia el diccionario y se serializa la nueva versión
        nuevo_diccionario = self.__dict__.copy()
        nuevo_diccionario.update({"messaggio": messaggio})
        return nuevo_diccionario


def cargar_obras(ruta_obras):
    """

    Funcion que carga las obras con las características pedidas,
    y luego entrega la lista de Obras.
    :param ruta_obras: Path de archivo de obras a cargar

    NOTA: DEBE USARSE la funcion obras_hook() como
     object hook para hacer el filtrado

    """
    with open(ruta_obras, encoding="utf-8") as archivo_obras:
        return json.load(archivo_obras, object_hook=obras_hook)


def obras_hook(dict_obras):
    """
    Object hook que hace los objetos de la clase Obra y los añade a una lista.
    HINT: Utilizar aquí el archivo caratteristicas ;)
    """
    with open(CARATTERISTICAS_CAMMINO) as archivo_caracteristicas:
        lista_caracteristicas = json.load(archivo_caracteristicas)
    # Se filtran los parametros del archivo de caracteristicas
    diccionario_filtrado = {c: dict_obras[c] for c in lista_caracteristicas}
    
    # Se desempaqueta el diccionario y se pasan las llaves como kwargs
    return Obra(**diccionario_filtrado)


def generar_mensaje(lista_obras):
    """
    Serializa las obras y las guarda en archivos
    :param lista_obras: Lista de objetos de tipo obra cargados.
    """
    # Se crea la carpeta 'Obras' si es que no existe
    os.makedirs("Obras", exist_ok=True)
    
    # Se serializa cada obra
    for obra in lista_obras:
        nombre_archivo = f"{obra.nome}-{obra.autore}.opera"
        with open(os.path.join("Obras", nombre_archivo), "wb") as archivo_obra:
            pickle.dump(obra, archivo_obra)


# Código principal
print("Cargando las obras...")
lista_obras = cargar_obras(OPERAS_CAMMINO)
print()
print("Codificando los mensajes (toma unos segundos)...")
generar_mensaje(lista_obras)
print()
print("Listo :)")

Cargando las obras...

Codificando los mensajes (toma unos segundos)...

Listo :)


## Ejercicio 4: Expresiones regulares

El profesor Vicente es un muy buen baterista, y un día le llego una canción de un alumno escrita de una manera muy peculiar...

```
BUMpatapatapatatss;-;BUMBUMpatatsspatatss;-;BUMpataBUMpataBUM;-;BUMtsstssBUMpata...
```

Él, ingeniosamente, descubrió qué significa cada elemento y cómo asociarlo a su representación en una notación más amigable:

- `BUM`: Bombo, que se representa con `O`
- `pata`: Caja, que se representa con `././`
- `tss`: Platillo, que se representa con `x/`
- `;-;`: Esto representa una barra de compás ` | `

Como favor, te pide que crees la función `traducir_pata(cancion)` que use *RegEx* para reemplazar los elementos de la canción por la nomenclatura que te presenta. Además, te deja un ejemplo:

```
BUMpatapatapatatss;-;BUMBUMpatatsspatatss;-;BUMpataBUMpataBUM;-;BUMtsstssBUMpata
```

La función `traducir_pata` debería retornar:

```
O ././ ././ ././ x/ | O O ././ x/ ././ x/ | O ././ O ././ O | O x/ x/ O ././ 
```

In [4]:
import re

def traducir_pata(cancion):
    # Aqui deberás hacer los cambios correspondientes
    cancion = re.sub("BUM", "O ", cancion)
    cancion = re.sub("pata", "././ ", cancion)
    cancion = re.sub("tss", "x/ ", cancion)
    cancion = re.sub(";-;", '| ', cancion)
    return cancion

print(traducir_pata("BUMpatapatapatatss;-;BUMBUMpatatsspatatss;-;BUMpataBUMpataBUM;-;BUMtsstssBUMpata"))

O ././ ././ ././ x/ | O O ././ x/ ././ x/ | O ././ O ././ O | O x/ x/ O ././ 
